# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio, pickle, os, ast
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
# mapping to the modules that make the app
sys.path.insert(0, "../../app")
sys.path.insert(0, "../../functions/popgrowth")

sys.path.insert
import models

c = models.CosmosdbClient()

In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty']}

Time is updated by the `time` function. It is always running.

In [4]:
c.run_query("g.V().hasLabel('time').valueMap()")
params['time'] = c.clean_nodes(c.res)[0]
params['time']

{'currentTime': 25797,
 'updatedFrom': 'azfunction',
 'updatedAt': '2023-03-15 12:40:00.675402',
 'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

## Population Resource Consumption

In [6]:
from tools import consumption

In [7]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)
pops_df

,name,objid,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,Rinmayherdroegaonbad Pestlan,2065545354087,0.719,0.445,0.484,0.601,0.50,5700878531117,0.5425,0.4938,0.722,true,BillmanLocal,pop,2065545354087
1,Uniabrafalefanak Batheights,2297177711622,0.549,0.425,0.319,0.711,0.50,1948134031021,0.5150,0.4700,0.724,true,BillmanLocal,pop,2297177711622
2,Ping Har,6116290971359,0.537,0.541,0.721,0.944,0.50,7750107587129,0.8325,0.6368,0.736,true,william.jeffrey.harding@gmail.com,pop,6116290971359
3,Bengarford Ge,8751304618851,0.373,0.636,0.487,1.031,0.50,3297014580825,0.7090,0.5725,0.311,true,william.jeffrey.harding@gmail.com,pop,8751304618851
4,Cantadt West,7678594134970,0.666,0.473,0.515,0.633,0.50,7123108087598,0.5240,0.3485,0.627,true,BillmanLocal2,pop,7678594134970
5,Cantadt Ho,8027372513635,0.552,0.583,0.641,0.577,0.50,7123108087598,0.5590,0.5160,0.574,true,BillmanLocal2,pop,8027372513635
6,Yuri War,1941048076759,0.451,0.439,0.617,0.342,0.50,0063686827678,0.4295,0.3342,0.372,false,BillmanLocal2,pop,1941048076759
7,Cantadt Gonrean,0504248736284,0.464,0.583,0.584,0.792,0.50,7123108087598,0.6380,0.5105,0.538,true,BillmanLocal2,pop,0504248736284
8,Cantadt Kha,0366602777869,0.549,0.560,0.586,0.551,0.50,7123108087598,0.5435,0.5018,0.491,true,BillmanLocal2,pop,0366602777869
9,Yuri Astau,5010780035102,0.446,0.361,0.541,0.441,0.50,0063686827678,0.4660,0.3635,0.234,true,BillmanLocal2,pop,5010780035102


In [8]:
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Belneurei,7635975712366,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal,species,7635975712366
1,Belneurei,7635975712366,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal,species,7635975712366
2,Tlasi,9841299620766,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,9841299620766
3,Tlasi,9841299620766,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,9841299620766
4,Pin,7609897667067,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,7609897667067
5,Pin,7609897667067,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,7609897667067
6,Pin,7609897667067,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,7609897667067
7,Pin,7609897667067,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,7609897667067
8,Pin,7609897667067,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,7609897667067
9,Pin,7609897667067,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,7609897667067


In [9]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,type,isHomeworld,username,objtype,id
0,Dos,terrestrial,7696190212701,1.071,0.741,1.000,7627175892791,Ramsartau,True,True,terrestrial,True,BillmanLocal,planet,7696190212701
1,Dos,terrestrial,7696190212701,1.071,0.741,1.000,7627175892791,Ramsartau,True,True,terrestrial,True,BillmanLocal,planet,7696190212701
2,Losmo,terrestrial,1831814248083,0.266,0.624,0.977,4328232676270,Am,True,True,terrestrial,True,william.jeffrey.harding@gmail.com,planet,1831814248083
3,Losmo,terrestrial,1831814248083,0.266,0.624,0.977,4328232676270,Am,True,True,terrestrial,True,william.jeffrey.harding@gmail.com,planet,1831814248083
4,Kergnabervel,terrestrial,1136187907097,0.527,0.000,0.910,4258108753342,Unal,True,True,terrestrial,True,BillmanLocal2,planet,1136187907097
5,Kergnabervel,terrestrial,1136187907097,0.527,0.000,0.910,4258108753342,Unal,True,True,terrestrial,True,BillmanLocal2,planet,1136187907097
6,Kergnabervel,terrestrial,1136187907097,0.527,0.000,0.910,4258108753342,Unal,True,True,terrestrial,True,BillmanLocal2,planet,1136187907097
7,Kergnabervel,terrestrial,1136187907097,0.527,0.000,0.910,4258108753342,Unal,True,True,terrestrial,True,BillmanLocal2,planet,1136187907097
8,Kergnabervel,terrestrial,1136187907097,0.527,0.000,0.910,4258108753342,Unal,True,True,terrestrial,True,BillmanLocal2,planet,1136187907097
9,Kergnabervel,terrestrial,1136187907097,0.527,0.000,0.910,4258108753342,Unal,True,True,terrestrial,True,BillmanLocal2,planet,1136187907097


First getting the list of consumption for each pop.


In [10]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df

,location_id,consumes,pop,consumption
0,1136187907097,[Organic],7,14
1,1831814248083,[Organic],2,4
2,5517071204057,[Organic],7,14
3,5593524437383,[Organic],7,14
4,7696190212701,[Organic],2,4
5,8149854906906,[Organic],7,14


We expand that dataset to break open the list of people in each pop. 

In [11]:
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\anaconda3\envs\exoplanets\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


,location_id,consumes,pop,consumption,multi
0,1136187907097,Organic,7,14,True
1,1831814248083,Organic,2,4,True
2,5517071204057,Organic,7,14,True
3,5593524437383,Organic,7,14,True
4,7696190212701,Organic,2,4,True
5,8149854906906,Organic,7,14,True


Then we get a list of the resources available on that location. 

In [12]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['7696190212701'], 'name': ['Dos']},
   {'volume': [2656], 'objid': ['3854368291228'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['1831814248083'], 'name': ['Losmo']},
   {'volume': [1391], 'objid': ['9475184497649'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['1136187907097'], 'name': ['Kergnabervel']},
   {'volume': [374], 'objid': ['5174873116348'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['8149854906906'], 'name': ['Field']},
   {'volume': [-79], 'objid': ['4958106057205'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['5517071204057'], 'name': ['Zumen']},
   {'volume': [878], 'objid': ['6966447146498'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['5593524437383'], 'name': ['Varsucia

In [13]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

../../functions/popgrowth\tools\consumption.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consumption_df[consumption_df['remaining']<0]['remaining'] = -1


,location_id,consumes,pop,consumption,multi,available,remaining
0,1136187907097,Organic,7,14,True,374.0,360.0
1,1831814248083,Organic,2,4,True,1391.0,1387.0
2,5517071204057,Organic,7,14,True,878.0,864.0
3,5593524437383,Organic,7,14,True,408.0,394.0
4,7696190212701,Organic,2,4,True,2656.0,2652.0
5,8149854906906,Organic,7,14,True,-79.0,-93.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [52]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
dtype: object

In [15]:
validate_resources_updated_query = f"""
g.V().has('objid','1212171618802').out('hasResource').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pd.DataFrame.from_records(c.clean_nodes(c.res))


g.V().has('objid','1212171618802').out('hasResource').valueMap()



,volume,name,objid,description,username,objtype,id
0,864,Organic,1139592844940,bilogical material that can be consumed by pops,william.jeffrey.harding@gmail.com,resource,1139592844940
1,107,Common Minerals,4579244562521,Iron and other common material used in constru...,william.jeffrey.harding@gmail.com,resource,4579244562521
2,46,Rare Minerals,7328863888302,"lithium, silver and other rare minerals used i...",william.jeffrey.harding@gmail.com,resource,7328863888302
3,10419,Water,0620999397406,"H2O ready to be consumed, either frozen or in ...",william.jeffrey.harding@gmail.com,resource,0620999397406


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [16]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,1212171618802,Organic,2,4,True,868.0,864.0
1,1566812213349,Organic,2,4,True,728.0,724.0


In [17]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

,location_id,consumes,pop,consumption,multi,available,remaining


Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [18]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOfSpecies').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out

location_id    1212171618802
consumes             Organic
pop                        2
consumption                4
multi                   True
available              868.0
remaining              864.0
Name: 0, dtype: object


[{'labels': [['location'], ['pop'], ['species']],
  'objects': [{'objid': ['1212171618802'], 'name': ['Yepi']},
   {'name': ['Dorf Maymanessha'],
    'objid': ['3740655755079'],
    'health': [0.5],
    'username': ['william.jeffrey.harding@gmail.com']},
   {'name': ['Gordi'],
    'objid': ['0327650709394'],
    'consumes': ['[Organic]']}]},
 {'labels': [['location'], ['pop'], ['species']],
  'objects': [{'objid': ['1212171618802'], 'name': ['Yepi']},
   {'name': ['Behaleslaikon Pla'],
    'objid': ['7148617739848'],
    'health': [0.5],
    'username': ['william.jeffrey.harding@gmail.com']},
   {'name': ['Gordi'],
    'objid': ['0327650709394'],
    'consumes': ['[Organic]']}]}]

In [19]:
print(f"{len(out)} pops will starve in {x.location_id}")

2 pops will starve in 1212171618802


We are only going to starve the pop if the pop consumes that resource. 

In [20]:
for i in out:
    health = i['objects'][1]['health'][0]
    objid = i['objects'][1]['objid'][0]
    consumes = yaml.safe_load(i['objects'][2]['consumes'][0])
    all_death = 0

print(i, x.consumes in consumes)

{'labels': [['location'], ['pop'], ['species']], 'objects': [{'objid': ['1212171618802'], 'name': ['Yepi']}, {'name': ['Behaleslaikon Pla'], 'objid': ['7148617739848'], 'health': [0.5], 'username': ['william.jeffrey.harding@gmail.com']}, {'name': ['Gordi'], 'objid': ['0327650709394'], 'consumes': ['[Organic]']}]} True


health just goes down. 

In [21]:
starve_query = f"""
                g.V().has('objid','{objid}').property('health',{health-params['starve_damage']})
                """
print(starve_query)


                g.V().has('objid','7148617739848').property('health',0.45)
                


Let's pretend that the pop will die. IF they do, it will create a `death event`.

In [22]:
death_event = consumption.death_by_starvation_event(c.clean_node(i['objects'][0]),i['objects'][1],params)
death_event

{'objid': '9409138528410',
 'name': 'starvation',
 'label': 'event',
 'text': 'The population (Behaleslaikon Pla) inhabiting Yepi has died of starvation.',
 'visibleTo': 'william.jeffrey.harding@gmail.com',
 'time': 23763,
 'username': 'azfunction'}

We will also link that event to an appropriate `happendAt` so that we can see it when looking at events related to specific things. 

In [23]:
print(c.create_custom_edge(death_event, c.clean_node(i['objects'][0]), 'happenedAt').strip())

g.V().has('objid','9409138528410')
            .addE('happenedAt')
            .to(g.V().has('objid','1212171618802'))


## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 


In [24]:
from tools import growth


In [25]:
pops_df,species_df,locations_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.5,Monuevil Harreipia,8622793176589,0.453,0.508,0.501,0.446,9132659179734,0.4735,0.4908,0.447,true,BillmanLocal2,pop,8622793176589
1,0.5,Chan Lamauhal,9972182873463,0.499,0.676,0.516,0.506,2075919117794,0.5110,0.5935,0.606,true,BillmanLocal2,pop,9972182873463
2,0.5,Dorf Maymanessha,3740655755079,0.538,0.524,0.526,0.670,3600019733579,0.5980,0.5610,0.538,true,william.jeffrey.harding@gmail.com,pop,3740655755079
3,0.5,Behaleslaikon Pla,7148617739848,0.509,0.439,0.699,0.399,8774319370110,0.5490,0.4940,0.404,true,william.jeffrey.harding@gmail.com,pop,7148617739848


In [26]:
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Losfenly,5303083615604,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5303083615604
1,Losfenly,5303083615604,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,5303083615604
2,Gordi,0327650709394,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0327650709394
3,Gordi,0327650709394,[Organic],"[Organic waste, Plastics]",0.7,0.2,william.jeffrey.harding@gmail.com,species,0327650709394


In [27]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,type,isHomeworld,username,objtype,id
0,Tesebrid,terrestrial,1566812213349,1.019,0.00,2,2367018655305,Varhu,True,True,terrestrial,True,BillmanLocal2,planet,1566812213349
1,Tesebrid,terrestrial,1566812213349,1.019,0.00,2,2367018655305,Varhu,True,True,terrestrial,True,BillmanLocal2,planet,1566812213349
2,Yepi,terrestrial,1212171618802,1.268,0.99,1,4037124139478,Qaneu,True,True,terrestrial,True,william.jeffrey.harding@gmail.com,planet,1212171618802
3,Yepi,terrestrial,1212171618802,1.268,0.99,1,4037124139478,Qaneu,True,True,terrestrial,True,william.jeffrey.harding@gmail.com,planet,1212171618802


In [28]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
pops_df

,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id,roll,grow
0,0.5,Monuevil Harreipia,8622793176589,0.453,0.508,0.501,0.446,9132659179734,0.4735,0.4908,0.447,true,BillmanLocal2,pop,8622793176589,0.140471,True
1,0.5,Chan Lamauhal,9972182873463,0.499,0.676,0.516,0.506,2075919117794,0.5110,0.5935,0.606,true,BillmanLocal2,pop,9972182873463,0.496344,True
2,0.5,Dorf Maymanessha,3740655755079,0.538,0.524,0.526,0.670,3600019733579,0.5980,0.5610,0.538,true,william.jeffrey.harding@gmail.com,pop,3740655755079,0.638432,False
3,0.5,Behaleslaikon Pla,7148617739848,0.509,0.439,0.699,0.399,8774319370110,0.5490,0.4940,0.404,true,william.jeffrey.harding@gmail.com,pop,7148617739848,0.414724,True


In [29]:
reproducing_pops

,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.5,Monuevil Harreipia,8622793176589,0.453,0.508,0.501,0.446,9132659179734,0.4735,0.4908,0.447,true,BillmanLocal2,pop,8622793176589
1,0.5,Chan Lamauhal,9972182873463,0.499,0.676,0.516,0.506,2075919117794,0.5110,0.5935,0.606,true,BillmanLocal2,pop,9972182873463
3,0.5,Behaleslaikon Pla,7148617739848,0.509,0.439,0.699,0.399,8774319370110,0.5490,0.4940,0.404,true,william.jeffrey.harding@gmail.com,pop,7148617739848


In [30]:
nodes = []
edges = []
# Event edges must be uploaded separately, because the event nodes haven't been created yet.    
event_edges = []

In [32]:
for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    location = locations_df.loc[i].to_dict()
    child = growth.grow_pop(p,species,params,syllables)
    nodes.append(child)
    event = growth.population_growth_event(p, location,child,params)
    nodes.append(event)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isInFaction"], "label": "isInFaction"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOfSpecies"})
    event_edges.append(c.create_custom_edge(event,location,'happenedAt'))
    event_edges.append(c.create_custom_edge(p,event,'caused'))



In [36]:
p

{'health': 0.5,
 'name': 'Behaleslaikon Pla',
 'objid': '7148617739848',
 'conformity': 0.509,
 'literacy': 0.439,
 'aggression': 0.699,
 'constitution': 0.399,
 'isInFaction': '8774319370110',
 'industry': 0.549,
 'wealth': 0.494,
 'factionLoyalty': 0.404,
 'isIdle': 'true',
 'username': 'william.jeffrey.harding@gmail.com',
 'objtype': 'pop',
 'id': '7148617739848'}

In [35]:
growth.grow_pop(p,species,params,syllables)

{'health': 0.3,
 'name': 'Behaleslaikon Plaben',
 'objid': '2001620281634',
 'conformity': 0.429,
 'literacy': 0.343,
 'aggression': 0.621,
 'constitution': 0.305,
 'isInFaction': '8774319370110',
 'industry': 0.329,
 'wealth': 0.328,
 'factionLoyalty': 0.305,
 'isIdle': 'true',
 'username': 'william.jeffrey.harding@gmail.com',
 'objtype': 'pop',
 'id': '2001620281634',
 'label': 'pop'}

In [34]:
pd.DataFrame.from_records(nodes)


,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id,label,text,visibleTo,time
0,0.3,Monuevil Harreipiador,7465445793593,0.416,0.469,0.456,0.521,9132659179734,0.284,0.324,0.433,true,BillmanLocal2,pop,7465445793593,pop,NaN,NaN,NaN
1,0.3,Monuevil Harreipiate,3197398916113,0.386,0.478,0.420,0.384,9132659179734,0.284,0.365,0.387,true,BillmanLocal2,pop,3197398916113,pop,NaN,NaN,NaN
2,NaN,population growth,2264452032270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,The population (Monuevil Harreipia) inhabiting...,BillmanLocal2,23763.0
3,0.3,Chan Lamauhalhay,0245165221776,0.554,0.715,0.461,0.589,2075919117794,0.307,0.267,0.537,true,BillmanLocal2,pop,0245165221776,pop,NaN,NaN,NaN
4,NaN,population growth,8977074283212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,The population (Chan Lamauhal) inhabiting Tese...,BillmanLocal2,23763.0
5,0.3,Behaleslaikon Plare,3121628058607,0.539,0.528,0.719,0.415,8774319370110,0.329,0.221,0.334,true,william.jeffrey.harding@gmail.com,pop,3121628058607,pop,NaN,NaN,NaN
6,NaN,population growth,3717898925491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,The population (Behaleslaikon Pla) inhabiting ...,william.jeffrey.harding@gmail.com,23763.0


In [37]:
pd.DataFrame.from_records(edges)

,node1,node2,label
0,3197398916113,8622793176589,childOf
1,3197398916113,9132659179734,isInFaction
2,3197398916113,5303083615604,isOfSpecies
3,0245165221776,9972182873463,childOf
4,0245165221776,2075919117794,isInFaction
5,0245165221776,5303083615604,isOfSpecies
6,3121628058607,7148617739848,childOf
7,3121628058607,8774319370110,isInFaction
8,3121628058607,0327650709394,isOfSpecies


In [39]:
event_edges

["\n        g.V().has('objid','2264452032270')\n            .addE('happenedAt')\n            .to(g.V().has('objid','1566812213349'))\n        ",
 "\n        g.V().has('objid','8622793176589')\n            .addE('caused')\n            .to(g.V().has('objid','2264452032270'))\n        ",
 "\n        g.V().has('objid','8977074283212')\n            .addE('happenedAt')\n            .to(g.V().has('objid','1566812213349'))\n        ",
 "\n        g.V().has('objid','9972182873463')\n            .addE('caused')\n            .to(g.V().has('objid','8977074283212'))\n        ",
 "\n        g.V().has('objid','3717898925491')\n            .addE('happenedAt')\n            .to(g.V().has('objid','1212171618802'))\n        ",
 "\n        g.V().has('objid','7148617739848')\n            .addE('caused')\n            .to(g.V().has('objid','3717898925491'))\n        "]